Web scraping com Python

Requerimentos

```
pip install beautifulsoup4
pip install scrapy
pip install Pillow
pip3 install requests bs4 tqdm

On Windows:
conda install opencv 
conda install -c auto pytesseract 
tesseract for windows: https://github.com/UB-Mannheim/tesseract/wiki
```

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import urllib.request
import shutil
import re # for regular expressions, can be removed if no test is used
from tqdm import tqdm # to print progress bar
from urllib.parse import urljoin, urlparse
import requests
import os 
#teste
#html = urlopen("http://mallet.pr.gov.br/Site_mallet/materias/2020/03_marco/18_covid_oficio/18_covid_oficio.asp")
#res = BeautifulSoup(html.read(),"html5lib");    
#print(res.title)

In [2]:
# Functions
def is_valid(url): #Checks whether `url` is a valid URL.
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url): # Returns all image URLs on a single `url`
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [3]:
def download(url, pathname): # Downloads a file given an URL and puts it in the folder `pathname`
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

            
def main(url, path):
    # get all images
    imgs = get_all_images(url)
    for img in imgs:
        # for each image, download it
        download(img, path)
        
main("http://mallet.pr.gov.br/Site_mallet/materias/2020/03_marco/18_covid_oficio/18_covid_oficio.asp", "covid19_mallet_scrappedImages")

Extracting images: 100%|██████████████████████████████████████████████████████████| 193/193 [00:00<00:00, 14851.32it/s]


In [4]:
#OCR
from pytesseract import Output
# importing modules
import cv2
import pytesseract

#for windows only
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
"""
from tesseract2dict import TessToDict
td=TessToDict()
"""
image = cv2.imread('covid19_mallet_scrappedImages/0309.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
#cv2.imshow('threshold image', threshold_img) # display image
#cv2.waitKey(0) # Maintain output window until user presses a key
#cv2.destroyAllWindows() # Destroying present windows on screen

custom_config = r'-c preserve_interword_spaces=1 --oem 1 --psm 1' #configuring parameters for tesseract
# now feeding image to tesseract
details = pytesseract.image_to_data(threshold_img, output_type='data.frame', config=custom_config, lang='por')
#details = pytesseract.image_to_data(threshold_img, output_type='data.frame', config=custom_config, lang='por', pandas_config=None)

"""
word_dict=td.tess2dict(threshold_img,'out','outfolder')
text_plain=td.word2text(word_dict,(0,0,threshold_img.shape[1],threshold_img.shape[0]))
print(type(text_plain))
print(text_plain)
#print(type(word_dict))
#print(word_dict)
"""

#print(details.text)
dt = details.text # select columm
print(dt)

0                NaN
1                NaN
2                NaN
3                NaN
4                  E
5        CORONAVÍRUS
6                NaN
7              CASOS
8                 EM
9             MALLET
10               NaN
11          03/09/20
12               NaN
13               NaN
14      Notificados:
15               790
16               NaN
17      Monitorados:
18                14
19               NaN
20        Suspeitos:
21                11
22               NaN
23        Positivos:
24                52
25               NaN
26    Recuperados:47
27               NaN
28      Descartados:
29               727
30               NaN
31           Óbitos:
32                 2
33               NaN
34               NaN
35               NaN
36               so.
37               |";
38        Secretaria
39                da
40             Saúde
41               NaN
42      Informações:
43                 á
44         Mallet-PR
Name: text, dtype: object


In [59]:
import pandas as pd
#create empty dataframe
col_names = ['Notificados','Monitorados','Suspeitos','Positivos','Recuperados','Descartados','Obitos','Atuais','Data']
coviddf = pd.DataFrame(columns = col_names)

#seleciona itens 
des = dt[details["text"].str.contains("1|2|3|4|5|6|7|8|9|:",  na=False)]
print(des)

11          03/09/20
14      Notificados:
15               790
17      Monitorados:
18                14
20        Suspeitos:
21                11
23        Positivos:
24                52
26    Recuperados:47
28      Descartados:
29               727
31           Óbitos:
32                 2
42      Informações:
Name: text, dtype: object


In [196]:
#cria novo dict para conter novos dados
covidict = {"dDescartados": [], "dNotificados":[],"dMonitorados":[],"dSuspeitos":[],"dPositivos": [], "dRecuperados":[],"dObitos":[],"dAtuais":[],"dData":[] }
#covidict["dDescartados"].append({'Descartados':des.str.contains(":")})
covidict = covidict # resetar dicionario
covidict

{'dDescartados': [],
 'dNotificados': [],
 'dMonitorados': [],
 'dSuspeitos': [],
 'dPositivos': [],
 'dRecuperados': [],
 'dObitos': [],
 'dAtuais': [],
 'dData': []}

In [168]:
#transforma em dict
enum=enumerate(des)
enum
d=dict((i,j) for i,j in enum)
d

{0: '03/09/20',
 1: 'Notificados:',
 2: '790',
 3: 'Monitorados:',
 4: '14',
 5: 'Suspeitos:',
 6: '11',
 7: 'Positivos:',
 8: '52',
 9: 'Recuperados:47',
 10: 'Descartados:',
 11: '727',
 12: 'Óbitos:',
 13: '2',
 14: 'Informações:'}

In [202]:
keyList=sorted(d.keys())

for i,v in enumerate(keyList): # printa no loop os valores
    #print(d[keyList[i]]) # printa o valor de i pela chave
    
    if d[keyList[i]].find("Desca") == 0:
        if len(d[keyList[i]]) > 12: 
            bDesca = d[keyList[i]]
            bDesca = bRecupera.replace("Descartados:", "")
            covidict["dDescartados"].append(int(bDesca))
        else:
            bDesca = d[keyList[i+1]]
            covidict["dDescartados"].append(int(bDesca))
    
    if d[keyList[i]].find("Recupera") == 0:
        if len(d[keyList[i]]) > 12: 
            bRecupera = d[keyList[i]]
            bRecupera = bRecupera.replace("Recuperados:", "")
            covidict["dRecuperados"].append(int(bRecupera))
        else:
            bRecupera = d[keyList[i+1]]
            covidict["dRecuperados"].append(int(bRecupera))
##### ate aqui (trocar b2 e por )
    if d[keyList[i]].find("Notifica") == 0:
        if len(d[keyList[i]]) > 12: 
            bNotifica = d[keyList[i]]
            bNotifica = bNotifica.replace("Notificados:", "")
            covidict["dNotificados"].append(int(bNotifica))
        else:
            bNotifica = d[keyList[i+1]]
            covidict["dNotificados"].append(int(bNotifica))
 
    if d[keyList[i]].find("Monitora") == 0:
        if len(d[keyList[i]]) > 12: 
            bMonitora = d[keyList[i]]
            bMonitora = bMonitora.replace("Monitorados:", "")
            covidict["dMonitorados"].append(int(bMonitora))
        else:
            bMonitora = d[keyList[i+1]]
            covidict["dMonitorados"].append(int(bMonitora))
    
    if d[keyList[i]].find("Suspei") == 0:
        if len(d[keyList[i]]) > 10: 
            bSuspei = d[keyList[i]]
            bSuspei = bSuspei.replace("Suspeitos:", "")
            covidict["dSuspeitos"].append(int(bSuspei))
        else:
            bSuspei = d[keyList[i+1]]
            covidict["dSuspeitos"].append(int(bSuspei))
    
    if d[keyList[i]].find("Posit") == 0:
        if len(d[keyList[i]]) > 10: 
            bPosit = d[keyList[i]]
            bPosit = bPosit.replace("Positivos:", "")
            covidict["dPositivos"].append(int(bPosit))
        else:
            bPosit = d[keyList[i+1]]
            covidict["dPositivos"].append(int(bPosit))
 
    if d[keyList[i]].find("Óbit") == 0:
        if len(d[keyList[i]]) > 7: 
            bObitos = d[keyList[i]]
            bObitos = bObitos.replace("Óbitos:", "")
            covidict["dObitos"].append(int(bObitos))
        else:
            bObitos = d[keyList[i+1]]
            covidict["dObitos"].append(int(bObitos))
            
    if len(d[keyList[i]]) == 8:
        #print("Achei a data")
        bData = d[keyList[i]]
        #if "/|0" in bData:
        covidict["dData"].append(bData)
        
# calcula casos atuais
bAtuais = int(bNotifica) - int(bDesca) - int(bRecupera) - int(bObitos)
covidict["dAtuais"].append(int(bAtuais))

#print(keyList)
covidict

{'dDescartados': [727, 727, 727, 727, 727],
 'dNotificados': [790, 790, 790, 790, 790],
 'dMonitorados': [14, 14, 14, 14, 14],
 'dSuspeitos': [11, 11, 11, 11, 11],
 'dPositivos': [52, 52, 52, 52, 52],
 'dRecuperados': [47, 47, 47, 47, 47],
 'dObitos': [2, 2, 2, 2, 2],
 'dAtuais': [14, 14, 14, 14, 14],
 'dData': ['03/09/20', '03/09/20']}

In [ ]:
bData = d[keyList[0]]
covidict["dData"].append(bData)